# A/B Testing para Evaluar Estrategias de Prompting (OpenAI)

Este cuaderno es una versión del experimento original (con Gemini) adaptada para usar la API de OpenAI. Está diseñado para ejecutarse tanto en local (usando un archivo `.env`) como en Google Colab (usando `userdata.get` para obtener secrets).

## 1) Configuración de entorno (Local / Colab)

El cuaderno detecta automáticamente si se está ejecutando en Colab y carga la clave de la forma apropiada. Si trabajas localmente, asegúrate de tener un archivo `.env` con `OPENAI_API_KEY`.

Nota: ya tienes una `.env` en este directorio, así que no deberías necesitar cambios adicionales para ejecución local.

In [ ]:
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Intentar detectar Colab
IN_COLAB = False
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Cargar .env si existe (útil en local)
load_dotenv()

OPENAI_API_KEY = None
if IN_COLAB:
    # En Colab el usuario puede guardar secretos y recuperarlos con userdata.get
    try:
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    except Exception:
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
else:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise ValueError('OPENAI_API_KEY no encontrada. Coloca tu clave en .env o en Colab secrets con la key OPENAI_API_KEY')

print(f"✅ OPENAI key cargada. Entorno: {'Colab' if IN_COLAB else 'Local'}")

## 2) Inicializar cliente de OpenAI

Usamos la librería oficial `openai` (cliente moderno `OpenAI`). Si no la tienes instalada, instala con `pip install openai python-dotenv ipywidgets`.

In [ ]:
from openai import OpenAI

# Inicializar cliente pasando la api_key directamente para evitar dependencias del entorno
client = OpenAI(api_key=OPENAI_API_KEY)

print('✅ Cliente OpenAI inicializado')

## 3) Prompts: Variante A (Zero-shot) y Variante B (Few-shot)

In [ ]:
prompt_A = """Descripción de la práctica: Optimización del uso del agua en cultivos de secano mediante técnicas de conservación de humedad.

Palabras clave: conservación agua, secano, humedad suelo, optimización.

Temas relacionados:"""

prompt_B = """Descripción de la práctica: Implementación de rotación de cultivos para mejorar la salud del suelo y reducir plagas.

Palabras clave: rotación cultivos, salud suelo, manejo plagas, sostenibilidad.

Temas relacionados: Agroecología, Manejo Integrado de Plagas, Fertilización Natural, Diversificación Agrícola.


Descripción de la práctica: Uso de drones para monitoreo de cultivos y detección temprana de enfermedades.

Palabras clave: drones agricultura, monitoreo cultivos, detección enfermedades, agricultura precisión.

Temas relacionados: Teledetección Agrícola, Agricultura Inteligente, Sensores Remotos, Análisis de Imágenes Aéreas.


Descripción de la práctica: Optimización del uso del agua en cultivos de secano mediante técnicas de conservación de humedad.

Palabras clave: conservación agua, secano, humedad suelo, optimización.

Temas relacionados:"""

print('📝 Prompt A (Zero-shot) - Prácticas Agroindustria:')
print('-' * 50)
print(prompt_A)
print('\n📝 Prompt B (Few-shot) - Prácticas Agroindustria:')
print('-' * 50)
print(prompt_B)

## 4) Función get_response adaptada a OpenAI

La función usa el endpoint de chat completions y retorna el texto generado.

In [ ]:
def get_response(
    prompt, model="gpt-4o-mini", temperature=0.7, max_tokens=256
):
    """
    Genera una respuesta usando OpenAI Chat Completions.
    Retorna el texto del assistant.
    """
    try:
        # client.chat.completions.create retorna un objeto con choices
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        # Acceder al contenido de la primera choice correctamente
        content = response.choices[0].message.content
        return content
    except Exception as e:
        # Manejo básico de errores
        print("Error al generar respuesta:", e)
        return None

## 5) Ejecutar experimento: Generar respuestas para cada variante

In [ ]:
test_prompts = [prompt_A, prompt_B]
responses = []
num_tests = 5
model_to_use = (
    "gpt-4o-mini"  # Cambia por gpt-4o o gpt-4o-mini según disponibilidad y coste
)  # Cambia por gpt-4o o gpt-4o-mini según disponibilidad y coste

print('🚀 Iniciando experimento A/B Testing con OpenAI...')
for idx, prompt in enumerate(test_prompts):
    var_name = chr(ord('A') + idx)
    print(f'📊 Generando respuestas para Variante {var_name}...')
    for i in range(num_tests):
        resp_text = get_response(prompt, model=model_to_use)
        data = {
            'variante': var_name,
            'prompt': prompt,
            'respuesta': resp_text
        }
        responses.append(data)
        print(f'  ✓ Respuesta {i+1}/{num_tests} generada')
    print()
df = pd.DataFrame(responses)
df.to_csv('respuestas_openai.csv', index=False)

print('📄 Resumen del experimento:')
print(f'Total de respuestas generadas: {len(df)}')
print(df.head())

## 6) Interfaz de evaluación humana (widgets)

La interfaz es equivalente a la del cuaderno original. En Colab ipywidgets puede requerir instalación/soporte adicional; en Jupyter Notebook / JupyterLab funciona de forma nativa si tienes `ipywidgets`.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Cargar el CSV guardado (por si reiniciaste el kernel)
df = pd.read_csv('respuestas_openai.csv')
df = df.sample(frac=1).reset_index(drop=True)  # mezclar para evaluación ciega
df['feedback'] = pd.Series(dtype='float')
response_index = 0

def update_response():
    global response_index
    new_response = df.iloc[response_index]['respuesta']
    if pd.notna(new_response):
        response.value = "<p>" + str(new_response) + "</p>"
    else:
        response.value = "<p>Sin respuesta</p>"
    count_label.value = f"Respuesta: {response_index + 1}/{len(df)}"

def on_button_clicked(b):
    global response_index
    user_feedback = 1 if b.description == '\U0001F44D' else 0
    df.at[response_index, 'feedback'] = user_feedback
    response_index += 1
    if response_index < len(df):
        update_response()
    else:
        df.to_csv('resultados_openai.csv', index=False)
        print('\n✅ Prueba A/B completada. Resultados guardados en resultados_openai.csv')
        summary_df = df.groupby('variante').agg(cantidad=('feedback', 'count'), puntuacion=('feedback', 'mean')).reset_index()
        print(summary_df)

response = widgets.HTML()
count_label = widgets.Label()
update_response()
thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_up_button.on_click(on_button_clicked)
thumbs_down_button = widgets.Button(description='\U0001F44E')
thumbs_down_button.on_click(on_button_clicked)
button_box = widgets.HBox([thumbs_down_button, thumbs_up_button])
print('👇 Evalúa cada respuesta usando los botones:')
display(response, button_box, count_label)

## 7) Conclusiones y próximos pasos

- Incrementar el número de iteraciones para mayor poder estadístico
- Probar diferentes modelos (`gpt-3.5-turbo`, `gpt-4`, `gpt-4o`) para comparar coste/beneficio
- Guardar metadatos (temperatura, modelo) junto a cada respuesta para análisis posterior
- Probar la ejecución en Colab y en local para validar la carga de secrets